<a href="https://colab.research.google.com/github/Srigowri/GPULouvain/blob/main/NUMBA_LOUVAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Created on Sun Jun  6 19:16:05 2021

@author: srigowri


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from numba import vectorize,jit
import time
from numba import cuda
import numba


import pandas as pd
import scipy.sparse as ss
from numba.experimental import jitclass
# from numba import int32, float32
from numba import jitclass, types, typed

In [ ]:
MAXIMUM_PASSES = -1
MIN_MODULARITY = 0.0000001


# self.node2com = typed.List.empty_list(numba.types.uint32,self.N-1)
# print("here--2")
# self.degrees = typed.List.empty_list(numba.types.uint32,self.N-1)
# print("here--3")
# self.gdegrees = typed.List.empty_list( numba.types.uint32,self.N-1)
# print("here--4")
# self.internals = typed.List.empty_list(numba.types.uint32,self.N-1)
# print("here--5")
# self.total_weight = (np.sum(self.data) + np.sum(self.diagonal))//2
# print("here--6")
# self.loops = typed.List.empty_list(np.uint32,self.N-1)
# print("here--7")

# spec = [
#     ('resolution', numba.types.float64),     
#     ('weight_key', numba.types.unicode_type), 
#     ('diagonal',numba.types.int64[::1]),
#     ('indices',numba.types.uint64[::1]),
#     ('indptr',numba.types.uint64[::1]),
#     ('data',numba.types.float64[::1]),
#     ('N',numba.types.uint64),
#     ('count',numba.types.uint64),
#     ('node2com', numba.types.int64[::1]),
#     ('degrees', numba.types.int64[::1]),
#     ('gdegrees', numba.types.int64[::1]),
#     ('internals', numba.types.int64[::1]),
#     ('total_weight', numba.types.float64),
#     ('loops', numba.types.int64[::1]),
# ]
    


# @jitclass(spec)
class LouvainCSR():
    def __init__(self,resolution=1.0,weight_key='weight'):
        
        self.resolution = resolution
        self.weight_key = weight_key
        
            
    def init_graph_param(self,indices,indptr,data,diagonal):                

        self.diagonal = diagonal
        self.indices = indices
        self.indptr = indptr
        self.data = data
        self.N = self.indptr.size
        self.count = 0
        self.node2com = np.zeros(self.N-1,dtype = np.int64)
        self.degrees = np.zeros(self.N-1,dtype = np.uint64)
        self.gdegrees = np.zeros(self.N-1,dtype = np.uint64)
        self.internals = np.zeros(self.N-1,dtype = np.uint64)
        self.total_weight = np.float64((np.sum(self.data) + np.sum(self.diagonal))//2)
        self.loops = np.zeros(self.N-1,dtype = np.uint64)

            
        for node in range(self.N-1):
                    
            self.node2com[node] = self.count                        
            deg = np.sum(self.data[self.indptr[node] : self.indptr[node+1]])            
            self.degrees[self.count] = deg + self.diagonal[node]
            self.gdegrees[node] = deg + self.diagonal[node]
            self.loops[node] = self.diagonal[node]
            self.internals[self.count] = self.loops[node]
            self.count += 1
            
    
    def modularity(self):
        links = numba.float32(self.total_weight)
        result = 0.
        if links > 0: 
            for community in set(self.node2com):
                in_degree = self.internals[int(community)]
                degree =   self.degrees[int(community)]
                result += (in_degree * self.resolution / links) -  ((degree / (2. * links)) ** 2)
        return result

    @staticmethod
    def randomize(items):
        randomized_items = list(items)
        np.random.shuffle(randomized_items)
        # cp.random.shuffle(cp.array(randomized_items))
        return randomized_items

    
    def neighcom(self,node):
        """
        Compute the communities in the neighborhood of node in the graph given
        with the decomposition node2com
        """
        weights = np.zeros(self.node2com.size)
        neighbours = self.indices[self.indptr[node] : self.indptr[node+1]]
        attributes = self.data[self.indptr[node] : self.indptr[node+1]]
        for neighbor,edge_weight in zip(neighbours,attributes):
            if neighbor!=node:
                neighborcom = self.node2com[neighbor]
                weights[neighborcom] = weights[neighborcom] + edge_weight
        return weights

    
    def remove(self,node, community, weight):
        """ Remove node from community com and modify status"""
        self.degrees[community] = (self.degrees[community] - self.gdegrees[node])
        self.internals[community] = float(self.internals[community] - weight - self.loops[node])
        self.node2com[node] = -1

    
    def insert(self,node, community, weight):
        """ Insert node into community and modify status"""
        self.node2com[node] = community
        self.degrees[community] = (self.degrees[community] + self.gdegrees[node])
        self.internals[community] = float(self.internals[community] + weight + self.loops[node])

    
    def first_phase(self):
        """Compute one level of communities
        """
        modified = True
        nb_pass_done = 0
        new_mod = self.modularity()
        

        while modified and nb_pass_done != MAXIMUM_PASSES:
            cur_mod = new_mod
            modified = False
            nb_pass_done += 1
#            allnodes = LouvainCSR.randomize(range(self.N-1))
            for node in range(self.N-1):
                com_node = self.node2com[node]
                degc_totw = self.gdegrees[node] / (self.total_weight * 2.)  # NOQA    
                neigh_communities = self.neighcom(node)                
                remove_cost = - self.resolution * neigh_communities[com_node] + (self.degrees[com_node] - self.gdegrees[node]) * degc_totw
                               
                self.remove(node, com_node,neigh_communities[com_node])                
                best_com = com_node
                best_increase = 0
#                LouvainCSR.randomize(zip(range(self.N),neigh_communities)):
                nonzero_idx = np.nonzero(neigh_communities)[0]
                
                for com, dnc in zip(nonzero_idx,neigh_communities[nonzero_idx]):
                    incr = remove_cost + self.resolution * dnc - self.degrees[com] * degc_totw
                    
                    if incr > best_increase:
                        best_increase = incr
                        best_com = com
                
                self.insert(node, best_com,neigh_communities[best_com])
                
                if best_com != com_node:
                    modified = True
                    
            new_mod = self.modularity()
            if new_mod - cur_mod < MIN_MODULARITY:
                break

    
    def induced_graph(self,partition):

        ret = nx.Graph()
        ret.add_nodes_from(partition)
        seen = set()
        nodelist = set(ret.nodes())
        for node1 in range(self.N-1):
            neighbours = self.indices[self.indptr[node1] : self.indptr[node1+1]]                       
            attributes = self.data[self.indptr[node1] : self.indptr[node1+1]]
            com1 = partition[node1]
            for node2, edge_weight in zip(neighbours,attributes):  
                if (node1,node2) not in seen and (node2,node1) not in seen:
                    com2 = partition[node2]          
#                    print((com1,com2))
                    w_prec = ret.get_edge_data(com1, com2, {self.weight_key: 0}).get(self.weight_key, 1)
                    ret.add_edge(com1, com2, **{self.weight_key: w_prec + edge_weight})                                
                    seen.add((node1,node2))
                    
        csr = nx.convert_matrix.to_scipy_sparse_matrix(ret,nodelist=nodelist,weight=self.weight_key)#,ret.nodes())                                                 
        return csr
                           
    @staticmethod
    def renumber(partition):
        """Renumber the values of the dictionary from 0 to n
        """
        values = set(partition)
        target = set(range(len(values)))

        if values != target:
            # add the values that won't be renumbered
            renumbering = dict(zip(target.intersection(values),
                                target.intersection(values)))
            # add the values that will be renumbered
            renumbering.update(dict(zip(values.difference(target),
                                        target.difference(values))))
            
            for i in range(len(partition)):
                partition[i] = renumbering[partition[i]]
        
        return partition

    @staticmethod
    def partition_at_level1(dendograms, level):
        partition = dendograms[0].copy()
        for index in range(1, level + 1):
        
            for node, community in enumerate(partition):
                partition[node] = dendograms[index][community]

        return partition


def detect_communities(indices,indptr,data,diagonal,weight_key='weight',resolution=1.0):
#    if graph.is_directed():
#        raise TypeError("Modularity is undefined for directed graph")
#    
#    edges = graph.number_of_edges()
#    if edges == 0:
#        return {node: i for i, node in enumerate(graph.nodes())}
    
    dendograms = []      
    louvain1 = LouvainCSR(resolution,weight_key)    


    louvain1.init_graph_param(indices,indptr,data,diagonal)    
    louvain1.first_phase()    
    prev_modularity = louvain1.modularity()            
    partition = LouvainCSR.renumber(louvain1.node2com)    
    dendograms.append(partition)
    current_graph = louvain1.induced_graph(partition)
    louvain1.init_graph_param(current_graph.indices,current_graph.indptr,current_graph.data,current_graph.diagonal())


    while True:
        louvain1.first_phase()
        new_modularity = louvain1.modularity()  
        # print("modularity",new_modularity)        
        if abs(new_modularity - prev_modularity) < MIN_MODULARITY:
            break    
        prev_modularity = new_modularity
        partition = LouvainCSR.renumber(louvain1.node2com)
        dendograms.append(partition)
        current_graph = louvain1.induced_graph(partition)
        louvain1.init_graph_param(current_graph.indices,current_graph.indptr,current_graph.data,current_graph.diagonal())

    return LouvainCSR.partition_at_level1(dendograms,len(dendograms)-1)

In [ ]:
#@title
@jit(nopython=True,parallel = True)
def modularity(partition,indices,indptr,data,edges,all_communities):
    within_community_degree = np.zeros(all_communities.size)
    total_community_degree = np.zeros(all_communities.size)

    for i in range(indptr.size-1):
        neighbours = indices[indptr[i] : indptr[i+1]]
        attributes = data[indptr[i] : indptr[i+1]]
        community = partition[i]
        total_community_degree[community] += len(neighbours)
        for n,a in zip(neighbours,attributes):
            if partition[n]== community:
                within_community_degree[community] += (a if n == i else a /2)
    
#    mod = 0.0
#    for community in all_communities:
#        mod += ((within_community_degree[community]/(edges))-(total_community_degree[community]/(2*edges)) ** 2)
#    
#    
    mod = np.sum((within_community_degree/(edges))-(total_community_degree/(2*edges)) ** 2)
    return mod

**ZACHARY KARATE**

In [ ]:
type(csr_graph.indices)

numpy.ndarray

In [ ]:
len(csr_graph.data)

156

In [ ]:
g = nx.karate_club_graph()
csr_graph = nx.convert_matrix.to_scipy_sparse_matrix(g)
part = detect_communities(csr_graph.indices,csr_graph.indptr,csr_graph.data,csr_graph.diagonal())
print(part)
all_communities = np.array(list(set(part)))
edges= len(csr_graph.indices)//2
%timeit modularity(part,csr_graph.indices,csr_graph.indptr,csr_graph.data,edges,all_communities)

[1 1 1 1 3 3 3 1 0 1 3 1 1 1 0 0 3 1 0 1 0 1 0 2 2 2 0 2 2 0 0 2 0 0]


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


The slowest run took 158209.01 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 8.59 µs per loop


In [ ]:
def read_data_file_as_coo_matrix(filename='edges.txt'):
    "Read data file and return sparse matrix in coordinate format."
    data = pd.read_csv(filename, sep=' ', header=None, dtype=np.uint64)
    data = pd.concat([data,pd.DataFrame(list(zip(data[1],data[0])))],axis=0)
    
    rows = data[0]  # Not a copy, just a reference.
    cols = data[1]
    
    ones = np.ones(len(rows), np.uint64)
    
    matrix = ss.coo_matrix((ones, (rows, cols)))
    return matrix


def save_csr_matrix(filename, matrix):
    """Save compressed sparse row (csr) matrix to file.

    Based on http://stackoverflow.com/a/8980156/232571

    """
    assert filename.endswith('.npz')
    attributes = {
        'data': matrix.data,
        'indices': matrix.indices,
        'indptr': matrix.indptr,
        'shape': matrix.shape,
    }
    np.savez(filename, **attributes)

def load_csr_matrix(filename):
    """Load compressed sparse row (csr) matrix from file.

    Based on http://stackoverflow.com/a/8980156/232571

    """
    assert filename.endswith('.npz')
    loader = np.load(filename)
    args = (loader['data'], loader['indices'], loader['indptr'])
    matrix = ss.csr_matrix(args, shape=loader['shape'])
    return matrix


"Test data file parsing and matrix serialization."

coo_matrix = read_data_file_as_coo_matrix("facebook_combined.txt")
csr_graph = coo_matrix.tocsr()
# save_csr_matrix("facebook_csr.npz", csr_graph)
# loaded_csr_matrix = load_csr_matrix("facebook_csr.npz")
# assert (csr_graph != loaded_csr_matrix).nnz == 0

In [ ]:
# part = detect_communities(csr_graph)
# all_communities = np.array(list(set(part)))
# edges = len(csr_graph.indices)//2
%timeit modularityCP(part,csr_graph.indices,csr_graph.indptr,csr_graph.data,edges,all_communities)

1 loop, best of 5: 1.37 s per loop
